## Parte 1: SQL y Manipulación de Datos

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

import pandas as pd
import matplotlib.pyplot as plt

Using Kaggle's public dataset BigQuery integration.


In [ ]:
# Preguntas más populares

query = """
SELECT
    q.id AS question_id
    , q.title AS titulo
    , q.view_count AS conteo_vistas
    , COUNT(a.id) AS num_respuestas
    , q.score AS question_score
FROM bigquery-public-data.stackoverflow.posts_questions q
LEFT JOIN bigquery-public-data.stackoverflow.posts_answers a ON q.id = a.parent_id
GROUP BY 1, 2, 3, 5
ORDER BY 4 DESC, 5 DESC
LIMIT 100;"""

query_job = client.query(query)
results = query_job.to_dataframe()
results.head()

,question_id,titulo,conteo_vistas,num_respuestas,question_score
0,184618,What is the best comment in source code you ha...,3113666,518,360
1,406760,What's your most controversial programming opi...,301961,407,363
2,1995113,Strangest language feature,496025,320,972
3,9033,Hidden Features of C#?,740759,296,1473
4,1711,What is the single most influential book every...,1105637,214,1437


In [ ]:
# Top 5 de preguntas más vistas por año

query = """
WITH top_preguntas AS (
    SELECT
        id
        , title AS titulo
        , view_count AS conteo_vistas
        , EXTRACT(YEAR FROM creation_date) AS anio
        , ROW_NUMBER() OVER (PARTITION BY EXTRACT(YEAR FROM creation_date) ORDER BY view_count DESC) AS rn
    FROM bigquery-public-data.stackoverflow.posts_questions
)
SELECT
    id
    , titulo
    , conteo_vistas
    , anio
FROM top_preguntas
WHERE rn <= 5"""

query_job = client.query(query)
results = query_job.to_dataframe()
results.head()

,id,titulo,conteo_vistas,anio
0,176918,Finding the index of an item in a list,5582167,2008
1,332289,How do I change the size of figures drawn with...,5186388,2008
2,82831,How do I check whether a file exists without e...,4940426,2008
3,363681,How do I generate random integers within a spe...,4849559,2008
4,48198,How do I find out which process is listening o...,4796904,2008


In [ ]:
# Preguntas con más votos pero pocas respuestas

query = """
WITH preguntas_respuestas AS (
    SELECT
        q.id AS pregunta_id
        , q.title AS titulo_pregunta
        , q.creation_date AS pregunta_fecha
        , a.id AS respuesta_id
        , a.creation_date AS respuesta_fecha
        , a.score AS respuesta_score
    FROM bigquery-public-data.stackoverflow.posts_questions AS q
    LEFT JOIN bigquery-public-data.stackoverflow.posts_answers AS a ON q.id = a.parent_id
),
votos AS (
    SELECT
        post_id AS pregunta_id
        , COUNT(*) AS num_votos
    FROM bigquery-public-data.stackoverflow.votes
    GROUP BY 1
)
SELECT
    qa.pregunta_id
    , qa.titulo_pregunta
    , qv.num_votos
    , COUNT(qa.respuesta_id) AS num_respuestas
FROM preguntas_respuestas AS qa
LEFT JOIN VOTOS AS qv ON qa.pregunta_id = qv.pregunta_id
WHERE qv.num_votos IS NOT NULL
GROUP BY 1, 2, 3
ORDER BY qv.num_votos DESC, num_respuestas ASC
LIMIT 100
"""

query_job = client.query(query)
results = query_job.to_dataframe()
results.head()

,pregunta_id,titulo_pregunta,num_votos,num_respuestas
0,11227809,Why is processing a sorted array faster than p...,38504,27
1,927358,How do I undo the most recent local commits in...,32577,100
2,2003505,How do I delete a Git branch locally and remot...,25587,42
3,231767,"What does the ""yield"" keyword do?",18878,48
4,388242,The Definitive C++ Book Guide and List,16117,1
...,...,...,...,...
95,6470651,How can I create a memory leak in Java?,5692,60
96,1260748,How do I remove a submodule?,5680,36
97,4456438,"How to pass ""Null"" (a real surname!) to a SOAP...",5659,9
98,134845,"Which ""href"" value should I use for JavaScript...",5656,55


In [ ]:
# Insignias relacionadas con python

query = """
    SELECT
      name AS badge_name
      , COUNT(*) AS badge_count
    FROM bigquery-public-data.stackoverflow.badges
    WHERE tag_based = TRUE
    AND LOWER(name) LIKE '%python%'
    GROUP BY 1
    ORDER BY 1 DESC
    LIMIT 10
"""
query_job = client.query(query)
df = query_job.to_dataframe()

print(df.head())

plt.figure(figsize=(10, 6))
plt.bar(df['badge_name'], df['badge_count'], color='aqua')
plt.xlabel('Insignia')
plt.ylabel('Conteo')
plt.title("Insignias relacionadas con la palabra 'Python' en StackOverflow")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Tiempo promedio en el que una persona se tarda en responderte una pregunta en StackOverflow

query = """
    WITH tiempos AS (
        SELECT
          q.id AS question_id
          , q.creation_date AS fecha_pregunta
          , MIN(a.creation_date) AS primer_fecha_respuesta
          -- Tiempo en segundos de la fecha de respuesta y la fecha de pregunta
          , TIMESTAMP_DIFF(MIN(a.creation_date), q.creation_date, SECOND) AS tiempo_en_contestar
          , COUNT(a.id) AS num_respuestas
        FROM bigquery-public-data.stackoverflow.posts_questions AS q
        LEFT JOIN bigquery-public-data.stackoverflow.posts_answers AS a ON q.id = a.parent_id
        WHERE a.creation_date IS NOT NULL
        AND q.creation_date IS NOT NULL
        GROUP BY 1, 2
        ORDER BY 2 DESC
    )
    SELECT AVG(tiempo_en_contestar) AS promedio_tiempo_contestar
    FROM tiempos
"""
query_job = client.query(query)

results = query_job.result()
df = results.to_dataframe()

promedio_tiempo_contestar_segundos = df['promedio_tiempo_contestar'].iloc[0]

#divmod regresa cociente y residuo como tupla
promedio_minutos, promedio_segundos = divmod(promedio_tiempo_contestar_segundos, 60)
promedio_horas, promedio_minutos = divmod(promedio_minutos, 60)
promedio_dias, promedio_horas = divmod(promedio_horas, 24)

print(f"Tiempo promedio en el que una persona tarda en responder una pregunta en StackOverflow:")
print(f"En días: {promedio_dias:.0f} días, {promedio_horas:.0f} horas, {promedio_minutos:.0f} minutos y {promedio_segundos:.0f} segundos")

Tiempo promedio en el que una persona tarda en responder una pregunta en StackOverflow:
En días: 17 días, 10 horas, 42 minutos y 45 segundos


## Parte 2: Modelación de Datos y Diseño de DataWarehouse

2.1 Para esta sección se busca responder a las siguientes preguntas. Utiliza ejemplos concretos usando cualquier nube de tu elección:

1. ¿Cuál es la diferencia entre Data Lake y un Data Warehouse?

    R = Data Lake es una forma de almacenar datos estructurados o no estructurados, se almacenan datos crudos sin procesar, se utiliza cuando no tienes bien definido el análisis que harás, mientras que el Data Warehouse se almacenan datos estructurados, su acceso es más rápido, se utilizan ETL para la extracción transformación y carga de datos
2. ¿Cómo garantizarías que un DataWarehouse sea escalable y pueda manejar el crecimiento de los datos a lo largo del tiempo?

    R = Definiendo particionamientos con fechas o una forma de tener la data almacenada por tiempo o identificadores
3. ¿Cómo transformarías datos de fuentes operacionales en un formato adecuado para análisis?

    R = Una manera de empezar es haciendo ETL, realizar test de duplicidad, rellenar huecos en la data 
4. ¿Cómo implementarías y asegurarías la calidad de datos y gobernanza en un Data Warehouse?

    R = Algo que me ha funcionado es identificar siempre los datos sensibles para mantener una seguridad adecuada al momento de almacenar y disponibilizar la data. También teniendo estándares o nomenclaturas al guardar la data, tener metadatos, documentación (mapeo de datos, procedencia, es decir origen y destino de los datos)
5. ¿Qué buenas prácticas implementarías para asegurar la seguridad en los datos del Data Warehouse?

    R = Esta respuesta va aunada a la anterior. Identificar datos sensibles como nombres, direcciones, biométricos, para no comprometer la seguridad de una persona. Manejar cifrado de datos, permisos, etc.
6. ¿Quiénes serían los usuarios de cada uno y qué tecnología utilizarías en la nube para apoyarte en tu desarrollo?

    R = Usuarios de un Datalake podrían ser Científicos de datos e Ingenieros de Datos; Usuarios de un Data Warehouse serían analistas más enfocados al negocio o empresa, ellos utilizan datos más estructurados para toma de decisiones.
    Para Almacenamiento de AWS Buckes S3 y para Azure el Blob Storage
    Para Exploración de datos en AWS Redshift, Athena y para Azure, Data Factory para la automatización y ahora con este examen estoy aprendiendo BigQuery.

2.2 Imaginemos que tenemos que habilitar para usuarios analíticos los datos transaccionales de las ventas a través de internet de una CPG. Describe como realizarías este proceso y cómo almacenarías las tablas de hechos y dimensiones. (Puedes tomar un ejemplo de datos existente o inventar uno).

Un ejemplo hipotética es las ventas de Coca en diferentes regiones.

1. El primer paso es recabar y entender los requisitos del negocio. Por ejemplo realizar un análisis de las ventas por tipo de producto en las diferentes regiones.
2. Tener un mapeo de las fuentes de datos que se utilizarán. Se podría tener bases de datos transaccionales que detallan la info del producto, los clientes y las transacciones.
3. Poder identificar qué modelo de datos se va a utilizar, el que conozco es el estrella, donde existen tablas de hechos o facts y dimensiones.
    - Existiría una tabla que contenga la información sumarizada por fecha, región, tipo de producto y algún identificador del cliente. La columna que serviría como identificador, sería una compuesta por fecha + producto + cliente
    - La otra tabla dimensional podrían ser varias, alguna de ellas sería una tabla de productos que contenga: tipo de producto, costo, categoría; otra tabla dimensional sería una del tiempo que contenga: fecha, año, mes, día; otra tabla dimensional sería la de regiones que contenga: nombre de la región, CP, sensidad de población, sector, etc; y la última tabla dimensional podría ser la del cliente que contenga: tipo de cliente (atleta, no atleta), fecha de nacimiento (para obtener la edad), el ID, género etc.
3. Una vez teniendo facts y dimensiones, podríamos determinar el tipo de almacenamiento podría ser un Data Warehouse con datos ya estructurados, Para realizar un ETL, es decir, limpieza de datos, transformaciones
4. Por último, identificar qué tipos de usuarios tendrian acceso a la información, segmentar usuarios, como analistas e ingenieros de datos, y utilizar herramientas de análisis como Tableau, Looker, etc. para generar informes.

## Parte 3: Procesamiento de Datos y ETL/ELT

3.1 Para esta sección se busca responder a las siguientes preguntas, dando ejemplos sobre alguna nube de su conocimiento

1. ¿Qué diferencia hay entre ETL y ELT?

    R = En el ETL los pasos son: Extracción, Transformación y Carga, mientras que en el ELT son: Primero la Extracción, luego la Carga y al final la Transformación.

2. ¿Cómo automatizarías los pipelines? ¿Qué criterios tomas en cuenta para elegir el tipo de automatización?

    R = Primero tomaría en cuenta la escalabilidad, costos, facilidad de uso, monitoreo y gestión de errores.

    - Paso 1: Diseñar el pipeline, definiéndo las etapas desde la extracción de datos, transformaciones y carga.

    - Paso 2: Crear las tareas que realizará el pipeline, aquí se definen las ejecuciones de funciones específicas.
    
    - Paso 3: Programar la ejecución de las tareas.

    - Paso 4: Monitorear y gestionar los errores.

    Por ejempo si tengo que automatizar algo con AWS como herramienta podría:

    - Paso 1: Extracción de datos en S3, programar scripts en python o pyspark de acuerdo al volumen de datos y mediante Glue crear las tareas, Cargar las tablas finales en Redshift.

    - Paso 2: Confogurar los scripts o tareas mediante Step Functions de AWS ó airflow

    - Paso 3: en un archivo .yaml puedo agregar la función del cron para la ejecución de tareas.

    - Paso 4: Mediante CloudWatch se monitorean las ejecuciones.

3. ¿Cómo gestionarías la parametrización de pipelines para que manejen distintos escenarios para no fallar ante valores hardcodeados?

    R = Utilizando variables de entorno, por ejemplo en un archivo .env. También utilizando archivos de configuración, dentro un yaml para almacenar parámetros. Otro ejemplo podría ser los secrets de AWS para recuperar contraseñas y usuarios de forma segura.

4. ¿Cómo gestionarías los secretos/llaves que se utilizan en los distintos pipelines, estableciendo claramente los secretos entre ambientes?

    R = Utilizando por ejemplo AWS Secrets Manager, sé que en ese servicio se pueden aplicar políticas de acceso basado en roles. Aunque no lo he hecho, los he usado.

5. ¿Cómo gestionarías las dependencias para asegurar que los pipelines se ejecutan en el orden correcto?

    R = He utilizado AWS Step Functions para ello y también Airflow mediante los dags.

6. Imagina que se tienen 1000 pipelines productivos. ¿Cómo realizarías una gestión eficiente de manejo de errores?
    
    R = Mediante alertas y notificaciones automáticas informar sobre fallos de cada pipeline, solo he utilizado Amazon CloudWatch Logs. 

7. Si se tiene una alta cantidad de pipelines, ¿Cómo realizarías el monitoreo de pipelines para ver qué establecer alertas o ser notificado en caso de fallas?

    R = Podría crear un dashboard que centralice las alertas o fallas de los pipelines para visualizar la salud de cada uno de ellos. Hasta ahora, he utilizado Slack para mandar alertas de fallos dentro de Ariflow.

8. ¿Qué estrategia de loggeo seguirías para diagnosticar errores, ejecuciones de pipelines, triggers, etc?

    R = Se me ocurre incluir metadatos a los logs para facilitar la búsqueda de los errores, el id del pipeline, el timestamps de ejecución, el ambiente, etc. Tener un repositorio especial para solo logs.

9. ¿Cómo garantizarías un versionado de los pipelines para contar con un control de cambios?

    R = Utilizando repositorios Git ó Gitlab para el control de cambios y manejar Branch de diferentes tipos: branch para fix, features, etc.

10. ¿Cómo establecerías en qué casos realizar loads incrementales vs full loads de manera automática?
    
    R = Incrementales cuando la data histórica no cambia y además si es un gran volumen de datos, se recomendaría realizar cargas incrementales para reducir costos de procesamiento.
    Full loads cuando la data es cambiante, también cuando se necesita recalcular datos o se cambia la estructura de ellos.


3.2 Imagina que quieres ingestar datos desde una API que tiene precios de commodities. La autenticación se realiza a través de un Bearer Token.

- ¿Para este ejemplo emplearías un proceso ETL o un proceso ELT? 
- Diseña un diagrama de flujo para el proceso de ETL/ELT elegido y explique cada paso del proceso y su función y otras cuestiones relevantes de cómo realizarías el pipeline en la nube.

Debido a que por la naturaleza de los datos de la API, los precios pueden ser muy cambiantes o se cambian con frecuencia, se podría utilizar un proceso ELT para realizar estas extracciones rápido dentro de un ambiente raw.

<img src="FlujoPipeline.png">

- Paso 1: Existe la API con auth Bearer Token.

- Paso 2: En un script de python se llama a la API mediante la librería de 'requests'.

- Paso 3: En ese mismo script del paso 2, se crea una instancia de boto3 para almacenar la data en un bucket de S3.

- Paso 4: En un script con pyspark se crea una Spark Session, leyendo la ubicación de la data en S3 y se realizan las limpiezas y transformaciones necesarias para almacenar los datos como parquet en otra ubicación de S3.

- Paso 5: En un script mediante BigQuery se almacena la data en una uri lista para ser consumida.

- Paso 6: Se llama a la tabla almacenada previamente en el paso 5 desde Looker, Tableau o cualquier herramienta de BI.

## Parte 4: Habilitación de datos para Analítica, visualización y reporteo

Para esta sección supongamos que se tiene una página web de comercio electrónico de una CPG. Dado que se registran distintos eventos de la interacción de los usuarios en las páginas web, hay una volumetría muy alta. 

4.1 Se busca responder a las siguientes preguntas. Proporciona ejemplos sobre alguna nube de su conocimiento:

- ¿Cómo prepararías y optimizarías los datos para mostrarlos eficientemente dentro de un dashboard?

    1. Utilizaría alguna herramienta de Amazon para la ingesta de datos en tiempo real desde la página web, ó Kafka que sé que se utiliza para streaming, aunque nunca lo he usado.

    2. Se almacenan los datos en S3

    3. Utilizaría AWS Glue para realizar las transformaciones con Pyspark

    4. Se almacenaría la data dentro de Redshift o Athena con datos particionados por fecha

- ¿Cómo garantizarías la calidad e integridad de los datos antes de que se carguen en el dashboard?

    Aplicando Data Quality, asegurando que la data venga completa y mediante Glue limpiarla, teniendo formato adecuado de fechas, enteros, cadenas, flotantes. Para el linage de datos existe AWS Lake Formation.

- ¿Qué estrategias seguirías en conjunto con los desarrolladores de tableros para garantizar que el tablero se mantenga ágil y responda rápidamente ante la creciente volumetría?
        
    Mediante Athena podría tener consultas interactivas y más rápidas. También podría optimizar las consultas de SQL para que no hayan cargas masivas tardadas, aplicar indexaxión, particionamientos. Utilizaría vistas de las tablas también.
    
- ¿Cómo gestionarías para no mostrar datos sensibles dentro del tablero?

    Tendríamos que configurar políticas de seguridad con AWS Identity and Access Management para restringir accesos. También puedo enmascarar la data mediante código en Glue para ocultar email, direcciones, leyendo desde el Data Catalog de AWS y haciendo transformaciones de la data para escribir los datos ya enmascarados.

    

4.2 Usando el ejemplo de comercio electrónico de la CPG, ¿Qué habilitarías a nivel técnico a personas analíticos y desarrolladores de Dashboards? Explica a detalle que diseño emplearías para satisfacer las necesidades de los usuarios que permitan garantizar el rendimiento, consistencia con la información del DataWarehouse de la empresa y que tenga un tamaño adecuado a la necesidad. Explica como serían los métodos de consumo propuestos.

1. Almacenamiento

    - Data Lake de Amazon en S3: Aquí se almacenarían los datos crudos, provenientes de páginas web, transacciones, registros de eventos como de clicks etc.
    Los datos se particionarían por fecha para facilitar el acceso y consultas. Ejemplo:

        - s3://mi-bucket/datalake/usuarios/2024/07/15/
        
        - s3://mi-bucket/datalake/transacciones/2024/07/15/

        - s3://mi-bucket/datalake/eventos/2024/07/15/
    
    - Data Warehouse en Redshift: Se almacenan los datos transformados y optimizados para el consumo de analistas. Se utilizaría un esquema de copos de nieve con tablas de facts o hechos (transacciones, eventos) y dimensiones (usuarios, productos).

2. Ingesta

    - Kinesis: Crear un flujo de datos de Kinesis para capturar eventos en tiempo real, y configurarlo para almacenar la data en S3. Aquí se capturan los clicks, los views de los usuarios etc.
    - AWS Glue para el procesamiento y limpieza de los datos, se emplean jobs para procesar los datos y se almacenen en Redshift.


3. Catalogo

    - AWS Glue Data Catalog donde se gestiona y se tienen los metadatos de la data que se almaceno previamente en S3 y Redshift. Aquí se definen las políticas de acceso para los usuarios. 

4. Herramientas de consumo de datos
    - Amazon Redshift para que los analistas ejecuten consultas en SQL, por ejemplo para generar informes mensuales de ventas, análisis sobre el comportamiento de los usuarios
    - Amazon Athena para desarrollo de dashboards interactivos y visualizaciones, esta herramienta es más quickSight
    

<img src="FlujoPipeline2.png">